<a href="https://colab.research.google.com/github/wildanputra/deeplearningtutorial/blob/master/surface_defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surface defect detection

sources inspiration:
https://towardsdatascience.com/deep-learning-computer-vision-and-automated-optical-inspection-774e8ca529d3

dataset : http://faculty.neu.edu.cn/yunhyan/NEU_surface_defect_database.html

in this case we will use 2 different method transfer learning for the classification, we will use pre-training & fine tuning with inception v3 model.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TensorFlow version is  1.14.0
